En esta parte se utilizará otro conjunto muy conocido de datos para aprendizaje, que consiste también de textos en inglés. Se trata de un conjunto de noticias de prensa de la Agencia Reuters, *Reuters-21578 *, https://kdd.ics.uci.edu/databases/reuters21578/reuters21578.html

Las noticias han sido categorizadas a mano, con etiquetas de varios tipos, incluyendo *temas, lugares, organizaciones, personas* y otros criterios. Están en formato SGML, las etiquetas aparecen embebidas en el texto. Existe documentación en el archivo README de la distribución.

Nos interesaremos en el tipo de etiqueta *temas (en inglés en los textos topics)*. El objetivo es nuevamente aprender los temas a partir de los textos de las noticias, con la diferencia de que este es un problema multi-etiqueta : cada noticia puede tener varios temas. En vez de resolver el problema multi-etiqueta inicial (con más de 100 tópicos distintos) les pedimos que lo transformen según las siguientes simplificaciones: considere solo los 3 temas más frecuentes, y transforme el problema multi-etiqueta en 3 problemas de clasificación binaria

No se realiza para esta parte una especificación detallada, sino que les pedimos a Uds. que armen los pasos de una solución y definan el detalle del notebook para esta parte. Mínimamente se espera que procesen la entrada SGML, y encuentren algún modo de enfocar el problema multi-etiqueta en la versión simplificada. Se debe proponer algún modo de tratar el texto, con eventuales mejoras respecto a la vectorización de la parte 1. Se deben aplicar clasificadores y medir su performance, mínimamente con precision, recall y medida-F. Finalmente, se espera una discusión detallada de todo lo realizado y eventuales propuestas de mejora.

Puntos a tener en cuenta del conjunto de datos:
* En Topics dentro de la etiqueta <REUTERS> puede haber:
    * YES: Tiene al menos una categoría(puede existir algún error y no tener ningúna categoria)
    * NO: No tiene categoría(puede existir algún error y tener alguna categoria, supuestamente en esta versión no sucede)
    * BYPASS: No está indexado, no se verificó la categoría
* Hay etiquetas que dividen el conjunto en train y test
* Etiquetas:
    * <TOPICS>, </TOPICS> [ONCE, SAMELINE]: Lista de categorias. Si hay alguna estan delimitadas por <D></D>.
    * <UNKNOWN>, </UNKNOWN> metadatos desconocidos
    * <TEXT>, </TEXT> cuerpo del texto

In [2]:
import glob
#Cargo los archivos como lista de strings en documentos
lista_nombres_archivo = glob.glob('reuters21578/*.sgm')
documentos = []
for nombre_archivo in lista_nombres_archivo:
    with open (nombre_archivo, "r") as archivo:
        documento = archivo.read()
        documentos.append(documento)

In [3]:
print documentos[0]

<!DOCTYPE lewis SYSTEM "lewis.dtd">
<REUTERS TOPICS="NO" LEWISSPLIT="TRAIN" CGISPLIT="TRAINING-SET" OLDID="14522" NEWID="9001">
<DATE>24-MAR-1987 15:59:17.10</DATE>
<TOPICS></TOPICS>
<PLACES><D>usa</D></PLACES>
<PEOPLE></PEOPLE>
<ORGS></ORGS>
<EXCHANGES></EXCHANGES>
<COMPANIES></COMPANIES>
<UNKNOWN> 
&#5;&#5;&#5;F
&#22;&#22;&#1;f2412&#31;reute
u f BC-ADVANCED-MAGNETICS-&lt;A   03-24 0066</UNKNOWN>
<TEXT>&#2;
<TITLE>ADVANCED MAGNETICS &lt;ADMG> IN AGREEMENT</TITLE>
<DATELINE>    CAMBRIDGE, Mass., March 24 - </DATELINE><BODY>Advanced Magnetics Inc said it
reached a four mln dlrs research and development agreement with
ML TEchnolgy Ventures LP, a limited partnership sponsored by
Merrill LYnch Capital Markets.
    Under the agreement, Advanced Magnetics will develop and
conducts clinical trials of its contrast agents for magnetic
resonance imaging.
    The agreement includes a warrant permitting MLTV to buy up
to 380,000 shares of Advanced Magnetics common stock through
February 1994 at 10

In [4]:
from BeautifulSoup import BeautifulSoup

raw_reuter_news = []
for documento in documentos:
    #coloco el contenido de los tags 'reuters' en otra lista: raw_reuter_news
    #esa lista contendra las noticias separadas en formato raw
    soup = BeautifulSoup(documento)
    for raw_reuter in soup('reuters'):
        raw_reuter_news.append(raw_reuter)

In [5]:
#verifico cantidad
print len(raw_reuter_news)

21578


In [9]:
raw_news_x_topics = {}
for raw_reuter_soup in raw_reuter_news:
    #para cada noticia
    for topics_soup in raw_reuter_soup.topics:
        #para cada topic en la lista de topics 
        #agrego la noticia a un diccionario clave topic y valor lista de noticias con ese topic.
        for topic_soup in topics_soup:
            try:
                #ya existe la lista.
                raw_news_x_topics[topic_soup].append(raw_reuter_soup)
            except KeyError:
                #es el primer valor
                raw_news_x_topics[topic_soup] = [raw_reuter_soup]

In [29]:
#ordeno el diccionario por cantidad de noticias en cada topic.
sorted_topics = sorted(raw_news_x_topics, key=lambda topic: len(raw_news_x_topics[topic]), reverse=True)
#los tres topics mas usados
primeros_topics = sorted_topics[:3]
for topic in primeros_topics:
        print topic,len(raw_news_x_topics[topic])

earn 3987
acq 2448
money-fx 801


In [65]:
#ya no necesito los metadatos
news_n_topics = []
for raw_reuter_soup in raw_reuter_news:
    #para cada noticia
    topics_de_la_noticia = []
    for topics_soup in raw_reuter_soup.topics:
        for topic_soup in topics_soup:
            #para cada topic en la lista de topics 
            topics_de_la_noticia.append(topic_soup)
    #veo si tiene las primeras categorias
    es_earn = 1 if u'earn' in topics_de_la_noticia else 0
    es_acq = 1 if u'acq' in topics_de_la_noticia else 0
    es_money_fx = 1 if u'money-fx' in topics_de_la_noticia else 0
    #armo una lista con body,titulo,fecha
    body = raw_reuter_soup.find('body').text if raw_reuter_soup.find('body') is not None else None
    title = raw_reuter_soup.find('title').text if raw_reuter_soup.find('title') is not None else None
    date = raw_reuter_soup.find('dateline').text if raw_reuter_soup.find('dateline') is not None else None
    contenido = (body,title,date)
    #guardo la tupla en la lista
    news_n_topics.append((contenido,es_earn,es_acq,es_money_fx))

In [66]:
#verifico cantidad noticias
print len(news_n_topics)
#verifico cantidad earn
print sum([a[1] for a in news_n_topics])
#verifico cantidad acq
print sum([a[2] for a in news_n_topics])
#verifico cantidad money-fx
print sum([a[3] for a in news_n_topics])

21578
3987
2448
801


In [68]:
print news_n_topics[0][0]

(u'Advanced Magnetics Inc said it\nreached a four mln dlrs research and development agreement with\nML TEchnolgy Ventures LP, a limited partnership sponsored by\nMerrill LYnch Capital Markets.\n    Under the agreement, Advanced Magnetics will develop and\nconducts clinical trials of its contrast agents for magnetic\nresonance imaging.\n    The agreement includes a warrant permitting MLTV to buy up\nto 380,000 shares of Advanced Magnetics common stock through\nFebruary 1994 at 10.50 dlrs per sahre.\n Reuter\n&#3;', u'ADVANCED MAGNETICS &lt;ADMG> IN AGREEMENT', u'CAMBRIDGE, Mass., March 24 -')
